In [1]:
import pandas as pd
import numpy as np

In [2]:
database_type = input('What data set wanna use: Small(0), Big(1)')
if(int(database_type)):
    ratings_type = 'ratings.csv'
else:
    ratings_type = 'ratings_small.csv'

movie_database = pd.read_csv('Data/movies_metadata.csv')
users_ratings = pd.read_csv('Data/' + ratings_type)


In [76]:
movie_database =  movie_database.rename(columns={'id':'movieId'})

# Get the total number of users and movies
num_users = len(pd.unique(users_ratings['userId']))
num_movies = len(pd.unique(movie_database['movieId']))

print("Total number of movies: " + str(num_movies))
print("Total number of users: " + str(num_users))

Total number of movies: 45436
Total number of users: 671


In [85]:
# Sample of ratings for user with ID = 1
users_ratings[users_ratings['userId'] == 1][0:5]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [90]:
def build_feedback_matrix(users_ratings,num_movies,num_users):
    # Initiate feedback matrix with all zero ratings
    # Get all unique users ids
    feedback_matrix = np.zeros((num_users,num_movies))
    users = pd.unique(users_ratings['userId'])
    for user_id in users:
        # Get the ratings for especific user id
        ratings = users_ratings[users_ratings['userId'] == user_id]
        num_ratings = len(ratings)
        # Set the rating of all the movies that the user Id has give rating
        for j in ratings.index:
            movie_id = ratings['movieId'].loc[j]
            if movie_id <= num_movies:
                feedback_matrix[user_id-1,movie_id-1] = ratings['rating'].loc[j]

    return feedback_matrix

In [91]:
# Build an NxM feedback matrix, N= Users, M= Movies
feedback_matrix = build_feedback_matrix(users_ratings,num_movies,num_users)

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [4. , 3.5, 0. , ..., 0. , 0. , 0. ],
       [0. , 5. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])